# Continuous integration

In this course we have been running tests on a single remote machine by invoking `pytest` from the command line. Every time we make changes to the code we manually re-run the tests in order to check that our changes haven't broken anything.

In a software project there might be many developers working on different parts the code remotely, then pushing their changes (commits) to a centralised repository, such as one on [GitHub](https://github.com). How can we make sure that tests are run whenever a new commit is received and that we are notified whenever a change breaks the code?

When running tests on your local machine you can be confident that the code is working in your environment, i.e. your operating system, version of python, etc. How can we be sure that the code will work on a range of environments?

[Continuous integration](https://en.wikipedia.org/wiki/Continuous_integration) (or CI) is the practice that attempts to solve these problems. The idea is that every time a new commit is pushed a CI server sets up a clean environment, does a `git clone` of your code from the central repository, then attempts to build it and run any tests.

There are many fantastic free CI services. In this course we will use [Travis](https://travis-ci.org) since it integrates well with [GitHub](https://github.com). (You should already have a GitHub account that is linked to Travis. If not, set one up now.)

## Configuring the CI build

Central to using travis is the `.travis.yml` file. This is a _hidden_ configuration file (as indicated by the `.` at the start of the file name) that sets options for the build. (The extension `.yml` indicates that this is a [YAML](https://en.wikipedia.org/wiki/YAML), which is a recursive acronym for "YAML Ain't Markup Language".)

A typical `travis.yml` file for a python build looks something like the following:

```yml
language: python

python:
    - "3.5"
    - "3.6"

script:
    - pytest
```

At the top of the file the `language` keyword is used to indicate that this is a `python` codebase. Next we specify which versions of python we want to use for our builds. Finally we come to the `script` section, which indicates the commands that will be run once the build environment is set up and our code has been cloned from GitHub (which will be done automatically). Full details about customising builds can be found [here](https://docs.travis-ci.com/user/customizing-the-build/).

(Note that there is an inconsistency with the naming of the `pytest` executable for the different versions of python provided by Travis. This makes it hard to use the same `script` section for a wide range of python versions. See [here](https://github.com/travis-ci/docs-travis-ci-com/issues/982) for details.)

## Your first CI build

We'll now walk through the process of running your first CI build. Before starting you'll need to open a _terminal_ by locating the `Home` Jupyter tab, clicking on the `New` dropdown button near the top right, then selecting `Terminal` from the list. For convenience, it might be preferable to split your screen so the terminal tab is next to this one. That way you can follow through the tutorial as you execute various commands.

First move into the `grid` directory. This has been set up with a `.travis.yml` file and a GitHub `README.md` file.

```bash
cd grid
```

Edit the `README.md` file so that both instances of `USERNAME` are replaced by your GitHub username.

Also edit the file `grid.py` to introduce a test-breaking bug. (Perhaps reintroduce the bug that you fixed earlier.) Run `pytest` to check that the tests fail.

```bash
pytest
```

Now initialise a new `git` repository.

```bash
git init
```

Add all of the files in the directory and stage them for the commit.

```bash
git add .
```

Commit the files that you've staged in your local repository.

```bash
git commit -m "First commit."
```

Go to your [GitHub](https://github.com) profile page and create a new repository called `ci-test`. To do so click on the `Repositories` tab, then the `New` button. Leave the check box for "Initialize this repository with a README" unchecked.

We now need to add Travis integration to the new repository. From the main page of the repository, click on the `Settings` tab, then select `Integrations & services` from the list on the left. On the `Services` drop down, scroll down the list unti you find `Travis CI` and select it. Accept the default settings. (Alternativley, you can go to your Travis homepage, and enable the CI hook there.)

Back on the command line we can now add a URL for the new remote repository that you just created. Once again, replace `USERNAME` with your actual GitHub username.

```bash
git remote add origin remote https://github.com/USERNAME/ci-skip
```

You can now push your commit to the remote repository.

```bash
git push origin master
```

Since we enabled a build hook for the `ci-skip` repository, Travis will detect the presence of the `.travis.yml` file in your new repository and automatically initiate a build. If you visit the URL for your new repository you will see a Travis _build status_ image on the main page. Clicking on this will take you to the Travis page for the repository, where you can see the progress of the current build, as well as the details of any previous builds.

You should see the status reported as _failed_. In addition, you might also receive an email notifying you of the error.

(It's normally bad practice to push code that you know is broken. In this case we're using it as an example to show how to go about fixing it the correct way.)

## Creating an issue

On the GitHub page for the `ci-test` repository you can open an _issue_ to alert that the code is broken. Click on the `Issues` tab followed by the `New issue` button. Give your issue whatever title you like, then hit submit.

## Pushing a fix

Back in your local repository fix the bug that you introduced earlier and verify that the tests now pass.

Having done this you can stage the `grid.py` file, then commit the change. For simplicity we'll do this in a single step.

```bash
git commit grid.py -m "Fixed a bug. [closes #1]"
```

Now push the commit to GitHub.

```bash
git push
```

The commit will now appear on GitHub and Travis will run another build against the updated version of the code. Once the build is complete you should hopefully see a green status badge on the repository homepage to indicate that it passed.

Take another look at the `Issues` tab. You should see that the issue that you opened is now _closed_. This happened automatically because we included the phrase "`closes #1`" somewhere in our commit message. Here `#1` is the issue number, i.e. the first issue that was opened.

(It is often helpful to put CI flags in brackets at the end of a commit message. This way it keeps the output of `git log` clean and it is easy to search for them.)

## Skipping a CI build

Sometimes you might commit changes that don't affect the functionality of the code, e.g. comments, or changes to the `README.md` file. In this case there is no need to run another CI build since none of the changes will affect the result of the tests. Since a build can be a time consuming process it would be wasteful to run one if wasn't absolutely necessary.

Thankfully there is a flag that can added to commit messages in order to indicate that a CI build should be skipped, `ci-skip`.

Edit the `README.md` file to include a new line saying "Testing is great!" (or whatever you prefer). Now commit your changes.

```bash
git commit README.md -m "Updated the README. [ci skip]"
```

Finally push the changes to GitHub.

```bash
git push
```

If you go to the GitHub or Travis page for your `ci-test` repository you should find that there wasn't a third CI build. On the GitHub page you can click on where it says "`3 commits`" to see the commit history. There should be a red cross (failed) next to the first commit, a green tick (passed) next to the second, and nothing (skipped) next to the third.